# Laboratorio 04 - Procesamiento de Lenguaje Natural (NLP)

Integrantes:
- José Pablo Kiesling Lange, 21581
- Melissa Pérez Alarcón, 21385

### Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb

### 1. Importación de datos
- Utilice el conjunto de datos IMDB proporcionado por Keras. pero esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes, utilice las 50.000 palabras más frecuentes.

In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=50000)

print(f"Tamaño del conjunto de entrenamiento: {len(x_train)}")
print(f"Tamaño del conjunto de prueba: {len(x_test)}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Tamaño del conjunto de entrenamiento: 25000
Tamaño del conjunto de prueba: 25000


### 2. Pre-procesamiento
- Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
- De las críticas, extraiga características (features) adicionales, por ejemplo. la longitud de la crítica, la proporción de palabras positivas/negativas y cualquier otra que considere pueda ser útil.

### 3. Modelo
- Cree un modelo LSTM que acepte las características (features) adicionales junto con la secuencia de palabras.
- Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y tal vez alguna capa densamente conectada después de la LSTM. (ver también la referencia al final de este documento)

### 4. Entrenamiento y Evaluación
- Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.